Visualisation of given dataset

In [21]:
import numpy as np
import h5py
def load_h5(h5_filename):
    f = h5py.File(h5_filename)
    data = f['data'][:]
    label = f['label'][:]
    return (data, label)
h5_filename = 'indoor3d_sem_seg_hdf5_data/ply_data_all_1.h5'
data_batch, label_batch = load_h5(h5_filename)
#dps = data_batch[0:80,:,0:6] # area1 conference1
#lbs = label_batch[0:80,:]
dps = data_batch[:,:,0:6] # area1 conference1
lbs = label_batch[:,:]

<ipython-input-21-8b1644b599b2>:4: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(h5_filename)


In [22]:
pts = dps[:,:,0:3]
cls = dps[:,:,3:6]
print(pts.shape)
print(cls.shape)
type(pts)
#all points in one place for visualisation purpose 
new_pts = pts.reshape((pts.shape[0]*pts.shape[1]),pts.shape[2])
new_cls = cls.reshape((cls.shape[0]*cls.shape[1]),cls.shape[2])

(1000, 4096, 3)
(1000, 4096, 3)


In [23]:
import open3d as o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(new_pts)
pcd.colors = o3d.utility.Vector3dVector(new_cls)
#o3d.io.write_point_cloud("./data.ply", pcd)
o3d.visualization.draw_geometries([pcd])


Model

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

import h5py
def load_h5(h5_filename):
    f = h5py.File(h5_filename)
    data = f['data'][:]
    label = f['label'][:]
    return (data, label)

# SOME_FILES = [line.rstrip() for line in open('indoor3d_sem_seg_hdf5_data/someFiles.txt')]#provider.getDataFiles('indoor3d_sem_seg_hdf5_data/all_files.txt')
# # Load ALL data
# data_batch_list = []
# label_batch_list = []
# for h5_filename in SOME_FILES:
#     data_batch, label_batch = load_h5(h5_filename)
#     data_batch_list.append(data_batch)
#     label_batch_list.append(label_batch)
# data_batches = np.concatenate(data_batch_list, 0)
# label_batches = np.concatenate(label_batch_list, 0)

data_batch, label_batch = load_h5('indoor3d_sem_seg_hdf5_data/ply_data_all_5.h5')
data_batches = data_batch[0:100,:,:]
label_batches = label_batch[0:100,:]
print(data_batches.shape)
print(label_batches.shape)
#cat_labels = tf.keras.utils.to_categorical(label_batches, num_classes = 13, dtype = "uint8")
X_train, X_test, Y_train, Y_test = train_test_split( data_batches, label_batches , test_size=0.2, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

input_image = keras.Input(shape = (4096,9,1))                                                            #B*N*9*1
num_point = 4096
x = Conv2D(64, [1,9], padding='VALID', strides = [1,1], activation = 'relu', name ='conv1')(input_image) #B*N*1*64
print(x.shape)
x = Conv2D(64, [1,1], padding='VALID', strides = [1,1], activation = 'relu', name ='conv2')(x)           #B*N*1*64
x = Conv2D(64, [1,1], padding='VALID', strides = [1,1], activation = 'relu', name ='conv3')(x)           #B*N*1*64
x = Conv2D(128, [1,1], padding='VALID', strides = [1,1],activation = 'relu', name ='conv4')(x)           #B*N*1*128
x= Conv2D(1024, [1,1], padding='VALID', strides = [1,1], activation = 'relu',name ='conv5')(x)           #B*N*1*1024
                                                                    #Now x is partial feature vector of each point
    # max pooling as a symmetric function is a key point to solve the cloud of disorder.
print(x.shape)
pc_feat1 = MaxPooling2D([num_point,1],strides=(2, 2), padding='VALID', name ='maxpool1')(x)   #maxpooling for each channel 
print(pc_feat1.shape)
pc_feat1 = Flatten()(pc_feat1)
print(pc_feat1.shape)
#model.add(Flatten())
pc_feat1 = Dense(256, activation = 'relu', name='fc1')(pc_feat1)        #global feature vectors through fully connected layers                                                      
pc_feat1 = Dense(128 , activation = 'relu', name='fc2')(pc_feat1)
print(pc_feat1.shape)
# CONCAT
a = tf.keras.layers.Reshape((1, 1, -1))(pc_feat1)                   #B*1*1*128
print(a.shape)

pc_feat1_expand = tf.tile(a, [1, num_point, 1, 1])                  #B*N*1*128
print(pc_feat1_expand.shape)
points_feat1_concat = tf.concat(axis=3, values=[x, pc_feat1_expand])#B*N*1*1152    (1024+128)axis = 3 implies along 4th column
print(points_feat1_concat.shape)
# CONV

y = Conv2D(512, [1, 1], padding='VALID', strides=[1, 1], activation = 'relu', name='conv6')(points_feat1_concat)
y = Conv2D(256, [1, 1], padding='VALID', strides=[1, 1], activation = 'relu', name='conv7')(y)
# tf.keras.layers.Dropout(rate, noise_shape=None, seed=None, **kwargs)

y = Dropout(rate=0.3)(y)                                                #rate = Fraction of the input units to drop.
y = Conv2D(13, [1, 1], padding='VALID', strides=[1, 1], activation = 'softmax', name='conv8')(y)#B*N*1*13
net = tf.squeeze(y, [2])                                                #B*N*13

# return net
model = keras.Model(inputs=input_image, outputs=net)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["sparse_categorical_accuracy"])
# loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True)

history = model.fit(X_train, Y_train, batch_size=20, epochs=10, verbose=1)     #verbose=0 will show you nothing (silent)
model.evaluate(X_test, Y_test, verbose=2)    #verbose=1 will show you an animated progress bar & verbose=2 will just mention the number of epoch
predictions = model.predict(X_test)
#pred = keras.utils.np_utils.probas_to_classes(model.predict(X_test))
pred = np.argmax(predictions,axis=2)
# plot loss during training
#pred = tf.nn.softmax(predictions)
# show the inputs and predicted outputs
# for i in range(10):
# 	print("actual=%s, Predicted=%s" % (Y_test[i], pred[i]))
print(pred.shape)
print(X_test.shape)
print(Y_test.shape)
print("Y_test[1] shape")
print(Y_test[1].shape)
print(pred[1].shape)
print("actual=%s, Predicted=%s" % (Y_test[1,:25], pred[1,:25]))
